In [1]:
import numpy as np
import pandas as pd
import re

import os
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import io
from tqdm.notebook import tqdm

from nltk.tokenize import word_tokenize
from collections import defaultdict 

In [2]:
import pickle

In [3]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [17]:
DATADIR = "D:\ImageCLEFphoto2008\annotations_complete_eng"
#DATADIR = '/Users/apple/Documents/SMAI Assignments/flda/annotations_complete_eng'
subfolders = os.listdir(DATADIR)
#subfolders = subfolders[1:]
texts_path = []
texts = []
path = ''
for subfolder in subfolders:

    path = DATADIR+"/"+subfolder+"/"
    texts = os.listdir(path)

    for textpath in texts:
        texts_path.append(path+textpath)

print(len(texts_path))

NotADirectoryError: [WinError 267] The directory name is invalid: 'D:/ImageCLEFphoto2008/annotations_complete_eng/.DS_Store/'

In [ ]:
np.random.seed(42)
np.random.shuffle(texts_path)  # Randomly shuffling the image names 
random_selection_text = np.random.choice(texts_path, 10000,replace=False)  # replace = False for unique paths

In [ ]:
random_selection_text[5687]

In [ ]:
DATASET_descriptions = []

annotation = None
desc = None

for i in tqdm(range(len(random_selection_text))):
    with open(random_selection_text[i], 'r', encoding='latin-1') as file:
        annotation = file.read()
        desc = [string.strip() for string in re.findall(r'<DESCRIPTION>(.*?)</DESCRIPTION>', annotation, re.DOTALL)]
        DATASET_descriptions.append(desc[0].replace(';',''))

In [ ]:
DATASET_descriptions[:10]

In [ ]:
len(DATASET_descriptions)

In [ ]:
data = DATASET_descriptions
 
sentences = []
word_set = set()
 
for sent in data:
    x = [i.lower() for  i in word_tokenize(sent) if i.isalpha()]
    sentences.append(x)
    for word in x:
        word_set.add(word)
#Set of vocab 
#word_set = set(word_set)
#Total documents in our corpus
total_documents = len(sentences)
 
#Creating an index for each word in our vocab.
index_dict = {} #Dictionary to store index for each word
i = 0
for word in word_set:
    index_dict[word] = i
    i += 1

In [ ]:
len(sentences)

In [ ]:
sentences[1]

In [ ]:
len(word_set)

NOTE: Ganesh, i am calling **`sentences`** as **`sentences_tokenized`** from here as you already tokenized them

In [ ]:
def calculateD_avg(sentences_tokenized):
    average_size = 0
    for sent in sentences_tokenized:
        average_size+=len(sent)
    average_size/= len(sentences_tokenized)
    return average_size

In [ ]:
def calculateFrequencyMatrix(sentences_tokenized):
    # Tells how many times does a term appear in the given sentence( in our case description )
    frequency_matrix = np.zeros((len(sentences_tokenized), len(word_set)))
    
    # Tells the no of documents in which the term appears
    document_frequency = np.zeros(len(word_set))
    
    for i in range(len(sentences_tokenized)):
        # Calculating the frequency of terms in sentence.
        for word in sentences_tokenized[i]:
            frequency_matrix[i, index_dict[word]]+=1
        
        # calculating the document frequency
        document_frequency += (frequency_matrix[i]>0)*1
    return frequency_matrix, document_frequency

In [ ]:
def calculateTermFrequencies(frequency_matrix, d_avg, sentences_tokenized):
    term_frequency = np.zeros(frequency_matrix.shape)
    for i in range(frequency_matrix.shape[0]):
        term_frequency[i] = (frequency_matrix[i]*(1.2+1))/ (frequency_matrix[i] + 1.2*(1 - 0.75 + (0.75*(len(sentences_tokenized[i])/d_avg))))
    return term_frequency

In [ ]:
def calculateInverseDocFrequency(document_frequency, total_no_of_docs):
    return np.log2( ((total_no_of_docs) - document_frequency + 0.5)/ (document_frequency + 0.5) )

In [ ]:
def calculateTfIdf(term_frequency, inv_doc_frequency):
    # copy the array so that we do not change term_frequency
    tf_idf = np.copy(term_frequency)
    for i in range(term_frequency.shape[0]):
        tf_idf[i]*= inv_doc_frequency
    return tf_idf

In [ ]:
frequency_matrix, document_frequency = calculateFrequencyMatrix(sentences)

In [ ]:
d_avg = calculateD_avg(sentences)

In [ ]:
term_frequency = calculateTermFrequencies(frequency_matrix, d_avg, sentences)

In [ ]:
inverse_doc_frequency = calculateInverseDocFrequency(document_frequency, len(sentences))

In [ ]:
tf_idf = calculateTfIdf(term_frequency, inverse_doc_frequency)

In [ ]:
with open('tfidfvisual.pkl','wb') as output:
    pickle.dump(tf_idf, output, pickle.HIGHEST_PROTOCOL)

In [ ]:
sentences[0]

In [ ]:
frequency_matrix[0,1270:1280] # position of 'a' in vocab is 1277, this will check if 'a' is repeated twice just like sentences[0]

In [ ]:
document_frequency

In [ ]:
#print(index_dict)

In [ ]:
#print(word_set)

In [ ]:
### To check whether the word is in the list or not 
ls = word_set
 
if any("eleven" in word for word in ls):
    print('\'eleven\' is there inside the list!')
else:
    print('\'eleven\' is not there inside the list')

In [ ]:
#Create a count dictionary
 
def count_dict(sentences):
    word_count = {}
    for word in word_set:
        word_count[word] = 0
        for sent in sentences:
            if word in sent:
                word_count[word] += 1
    return word_count
 
word_count = count_dict(sentences)

In [ ]:
print(word_count)

In [ ]:
class tfidfscorecalc:
    def __init__(self):
        # tf-idf scores of images
        self.tf_idf = []
        # Inverse document frequency of images
        self.inv_doc_freq = []
        self.k = 0
        
        
    def computeTfIdfScoredMSTD(self, texts):
        # contains the term frequency of each text
        document_frequency = np.zeros(self.k)

        tf_idf = []
        print("Calculating tf-idf scores ...")
        
        for i in tqdm(range(len(texts))):
            
            term_frequency = np.zeros(self.k)
            # Return labels
            visual_terms = self.getVisualTermsOfImage(texts[i])
            
            # Calculate the histogram
            for i in range(self.k):
                term_frequency[visual_terms[i]]+=1 
            # Add the elements that appear in this document
            document_frequency += (term_frequency>0)*1
            
            # K is 1.2 in Okapi formula. b is not necessary as the average size of documents is same as document size for all images
            term_frequency = (term_frequency*(1.2+1))/(term_frequency+1.2)
            tf_idf.append(term_frequency)
        
        inverse_document_frequency = np.log2((len(images)-document_frequency + 0.5)/(document_frequency+0.5))
        for i in range(len(images)):
            tf_idf[i] *= inverse_document_frequency
            
        self.tf_idf = tf_idf
        self.inv_doc_freq = inverse_document_frequency

In [ ]:
#Term Frequency
def termfreq(document, word):
    N = len(document)
    occurance = len([token for token in document if token == word])
    return occurance/N

In [ ]:
#Inverse Document Frequency
 
def inverse_doc_freq(word):
    try:
        word_occurance = word_count[word] + 1
    except:
        word_occurance = 1
    return np.log(total_documents/word_occurance)

In [ ]:
def tf_idf(sentence):
    tf_idf_vec = np.zeros((len(word_set),))
    for word in sentence:
        tf = termfreq(sentence,word)
        idf = inverse_doc_freq(word)
         
        value = tf*idf
        tf_idf_vec[index_dict[word]] = value 
    return tf_idf_vec

In [ ]:
def bag_of_words(sent):
    count_dict = defaultdict(int)
    vec = np.zeros(len_vector)
    for item in sent:
        count_dict[item] += 1
    for key,item in count_dict.items():
        vec[index_dict[key]] = item
    return vec

In [ ]:
vector = bag_of_words(sentences[109])
print(vector)

In [ ]:
import pickle

var = None
with open("tfidftextual.pkl", "rb") as file:
    pickle.load(file)
    
var.tf_idf